This file aims for standarizing the tags for products extend tags.

In [1]:
!pip install anthropic --upgrade
import pandas as pd
from tqdm import tqdm
import json
from google import genai
from google.genai import types
from dotenv import dotenv_values
from utils.llm_manager import LlmManager
import time 
import anthropic
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.messages.batch_create_params import Request

  Attempting uninstall: anthropic
    Found existing installation: anthropic 0.57.1
    Uninstalling anthropic-0.57.1:
      Successfully uninstalled anthropic-0.57.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-anthropic 0.2.4 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.3.1 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
classes = [
    "功能",
    "材質",
    "訴求",
    "使用情境",
    "風格",
    "族群"
]

## Momo

In [3]:
#
# * load in raw data
momo_products_extend = pd.read_excel("sources/文字型數據_momo.xlsx")

In [4]:
momo_products_extend.head()

,id,source_product_id,domain,category,subcategory,further_subcategory,extend_class,extend_subclass,extend_detail,extend_detail_raw,extend_unit,source,domain_from_classification,category_from_classification,subcategory_from_classification,further_subcategory_from_classification,match_key
0,33c78847-7bd5-4a78-8139-9b9f461b4fb3,momo_10029797,NaN,NaN,NaN,NaN,使用情境,NaN,家庭,家庭,NaN,momo,智慧家電,通訊,行動通訊設備,無線電話,momo_10029797_使用情境_None_家庭_None
1,e9fbc893-9549-4d3e-a038-db38f5b0f283,momo_10029797,NaN,NaN,NaN,NaN,使用情境,NaN,辦公室,辦公室,NaN,momo,智慧家電,通訊,行動通訊設備,無線電話,momo_10029797_使用情境_None_辦公室_None
2,04973514-0a3c-4a96-8685-6c24983a9f65,momo_10278628,NaN,NaN,NaN,NaN,使用情境,NaN,辦公室,辦公室,NaN,momo,智慧家電,通訊,行動通訊設備,無線電話,momo_10278628_使用情境_None_辦公室_None
3,8e808bee-715d-48b1-9233-630d4c871d55,momo_10278628,NaN,NaN,NaN,NaN,使用情境,NaN,家庭,家庭,NaN,momo,智慧家電,通訊,行動通訊設備,無線電話,momo_10278628_使用情境_None_家庭_None
4,31142e90-fc70-45cb-886c-0019ae638a91,momo_10278629,NaN,NaN,NaN,NaN,使用情境,NaN,高品質通話 收音清晰,高品質通話 收音清晰,NaN,momo,智慧家電,通訊,行動通訊設備,無線電話,momo_10278629_使用情境_None_高品質通話 收音清晰_None


In [5]:
momo_products_extend['extend_class'].unique()

array(['使用情境', '材質', '風格', '訴求', '功能', '材質_部件材質', '色彩', '族群'],
      dtype=object)

In [6]:
# 標準化類似的 extend class（材質_部件材質 -> 材質）
momo_products_extend['extend_class'] = (momo_products_extend['extend_class']
                                        .apply(
                                            lambda ext_class: "材質" if ext_class == "材質_部件材質" else ext_class
                                            )
                                        )

In [7]:
momo_products_extend['extend_class'].unique()

array(['使用情境', '材質', '風格', '訴求', '功能', '色彩', '族群'], dtype=object)

extend_detail 標準化：
1. 讀取對應的 extend class JSON 檔案
2. 針對每列 raw data，如果 extend detail 出現在任何一個 value 的 list 裡面，標準化的 detail 設定為 key
3. 若出現在 low quality tags 或是 remove tags 裡面，額外標註「low quality」

momo_tags = {
    "class": {
        "tags": ,
        "low_quality": 
    }   
}

In [31]:
momo_tags = {subclass: {"tags": {}, "low_quality": []} for subclass in classes}

for subclass in classes:
    with open(f"./sources/momo/final_{subclass}.json", "rb") as f:
        tags_json = json.load(f)
        for detail in tags_json:
            for tag in detail["aliases"]:      # * 翻轉 tag pair, 轉換成 aliases: standard_tag 的格式
                momo_tags[subclass]["tags"][tag] = detail["standard_tag"]
    with open(f"./sources/momo/low_quality_{subclass}.json", "rb") as f:
        momo_tags[subclass]["low_quality"] += json.load(f)
    with open(f"./sources/momo/removed_{subclass}.json", "rb") as f:
        for detail in json.load(f):
            momo_tags[subclass]["low_quality"] += detail["original_aliases"]


    # ***** 第二輪跑完的也放入
    if subclass != "材質":
        with open(f"./sources/momo/round_2/final_{subclass}.json", "rb") as f:          
            tags_json = json.load(f)
            for detail in tags_json:
                for tag in detail["aliases"]:      # * 翻轉 tag pair, 轉換成 aliases: standard_tag 的格式
                    momo_tags[subclass]["tags"][tag] = detail["standard_tag"]
        with open(f"./sources/momo/round_2/low_quality_{subclass}.json", "rb") as f:
            momo_tags[subclass]["low_quality"] += json.load(f)
        with open(f"./sources/momo/round_2/removed_{subclass}.json", "rb") as f:
            for detail in json.load(f):
                momo_tags[subclass]["low_quality"] += detail["original_aliases"]

    momo_tags[subclass]["low_quality"] = set(momo_tags[subclass]["low_quality"])
    
    

In [32]:
def standarizing_momo(row):
    # * 跳過 class == 色彩 先不處理
    if row["extend_class"] == "色彩":
        row["standarized_tag"] = row["extend_detail"]
        return row

    else:
        # * 嘗試取得標準化標籤
        standarized_tag = momo_tags[row['extend_class']]['tags'].get(row['extend_detail'], "N/A")

        # * 若取得失敗，從 low_quality 標籤中找
        if standarized_tag == "N/A":
            if row["extend_detail"] in momo_tags[row['extend_class']]['low_quality']:
                standarized_tag = "low_quality"
            # * 若 low_quality 裡面也沒有，則維持 N/A

        row["standarized_tag"] = standarized_tag
        return row
    

In [33]:
tqdm.pandas(desc = "standarizing momo products extend...")
momo_standarized = momo_products_extend.progress_apply(standarizing_momo, axis = 1)

standarizing momo products extend...: 100%|██████████| 696508/696508 [01:56<00:00, 5987.95it/s]


In [34]:
na_unique = len(momo_standarized[momo_standarized['standarized_tag'] == "N/A"]["extend_detail"].unique())
total_unique = len(momo_standarized['extend_detail'].unique())
coverage_rate = (total_unique - na_unique) / total_unique
print(f"標籤種類覆蓋率: {coverage_rate:.2%}")

# 計算每個類別中，unique 標籤的覆蓋率
coverage_by_class_unique = momo_standarized.groupby('extend_class').apply(
    lambda group: (group[group['standarized_tag'] != "N/A"]['extend_detail'].nunique()) / 
                  (group['extend_detail'].nunique()), include_groups = False
).sort_values(ascending=False)
coverage_by_class_unique

標籤種類覆蓋率: 93.54%


extend_class
色彩      1.000000
功能      0.999763
族群      0.998682
風格      0.998558
訴求      0.996349
使用情境    0.982417
材質      0.380453
dtype: float64

## Amazon

In [21]:
#
# * load in raw data
amazon_products_extend = pd.read_excel("sources/文字型數據_amazon.xlsx")

In [35]:
amazon_products_extend.head()

,id,source_product_id,domain,category,subcategory,further_subcategory,extend_class,extend_subclass,extend_detail,extend_detail_raw,extend_unit,source,domain_from_classification,category_from_classification,subcategory_from_classification,further_subcategory_from_classification,project_year,match_key
0,262eee05-b0af-4ec2-a711-d2cce5727c18,amazon_0545498562,NaN,NaN,NaN,NaN,訴求,NaN,award-winning craft kit for kids,award-winning craft kit for kids,NaN,amazon,母嬰兒童,兒童玩具,益智玩具,串珠/拼豆,113,amazon_0545498562_訴求_None_award-winning craft ...
1,a2f645d4-4340-46b4-9e69-58710e15ffec,amazon_0545498562,NaN,NaN,NaN,NaN,訴求,NaN,includes 9 vibrant clay colors,includes 9 vibrant clay colors,NaN,amazon,母嬰兒童,兒童玩具,益智玩具,串珠/拼豆,113,amazon_0545498562_訴求_None_includes 9 vibrant c...
2,090cccc4-8132-438c-865c-8a16b44678ce,amazon_0545498562,NaN,NaN,NaN,NaN,訴求,NaN,simple step-by-step instructions included,simple step-by-step instructions included,NaN,amazon,母嬰兒童,兒童玩具,益智玩具,串珠/拼豆,113,amazon_0545498562_訴求_None_simple step-by-step ...
3,4f44df21-411d-4a76-a615-266cd72e28d9,amazon_0545498562,NaN,NaN,NaN,NaN,訴求,NaN,create over 35 unique charms,create over 35 unique charms,NaN,amazon,母嬰兒童,兒童玩具,益智玩具,串珠/拼豆,113,amazon_0545498562_訴求_None_create over 35 uniqu...
4,73825118-2874-45f3-a099-3e92921bbd5d,amazon_0545498562,NaN,NaN,NaN,NaN,訴求,NaN,suitable for ages 8 and up,suitable for ages 8 and up,NaN,amazon,母嬰兒童,兒童玩具,益智玩具,串珠/拼豆,113,amazon_0545498562_訴求_None_suitable for ages 8 ...


In [37]:
amazon_products_extend['extend_class'].unique()

array(['訴求', '使用情境', '功能', '風格', '材質', '色彩', '族群'], dtype=object)

In [38]:
# 標準化類似的 extend class（材質_部件材質 -> 材質）
amazon_products_extend['extend_class'] = (amazon_products_extend['extend_class']
                                        .apply(
                                            lambda ext_class: "材質" if ext_class == "材質_部件材質" else ext_class
                                            )
                                        )

In [39]:
amazon_products_extend['extend_class'].unique()

array(['訴求', '使用情境', '功能', '風格', '材質', '色彩', '族群'], dtype=object)

extend_detail 標準化：
1. 讀取對應的 extend class JSON 檔案
2. 針對每列 raw data，如果 extend detail 出現在任何一個 value 的 list 裡面，標準化的 detail 設定為 key
3. 若出現在 low quality tags 或是 remove tags 裡面，額外標註「low quality」

amazon_tags = {
    "class": {
        "tags": ,
        "low_quality": 
    }   
}

In [63]:
amazon_tags = {subclass: {"tags": {}, "low_quality": []} for subclass in classes}
for subclass in classes:
    with open(f"./sources/amazon/final_{subclass}.json", "rb") as f:
        tags_json = json.load(f)
        for detail in tags_json:
            for tag in detail["aliases"]:      # * 翻轉 tag pair, 轉換成 aliases: standard_tag 的格式
                amazon_tags[subclass]["tags"][tag] = detail["standard_tag"]
    with open(f"./sources/amazon/low_quality_{subclass}.json", "rb") as f:
        amazon_tags[subclass]["low_quality"] += json.load(f)
    with open(f"./sources/amazon/removed_{subclass}.json", "rb") as f:
        for detail in json.load(f):
            if isinstance(detail["original_aliases"], list):
                amazon_tags[subclass]["low_quality"] += detail["original_aliases"]
            else:
                print(detail["original_aliases"])


    # ***** 第二輪跑完的也放入
    if subclass != "材質":
        with open(f"./sources/amazon/round_3/final_{subclass}.json", "rb") as f:          
            tags_json = json.load(f)
            for detail in tags_json:
                for tag in detail["aliases"]:      # * 翻轉 tag pair, 轉換成 aliases: standard_tag 的格式
                    amazon_tags[subclass]["tags"][tag] = detail["standard_tag"]
        with open(f"./sources/amazon/round_3/low_quality_{subclass}.json", "rb") as f:
            amazon_tags[subclass]["low_quality"] += json.load(f)
        with open(f"./sources/amazon/round_3/removed_{subclass}.json", "rb") as f:
            for detail in json.load(f):
                if isinstance(detail["original_aliases"], list):
                    amazon_tags[subclass]["low_quality"] += detail["original_aliases"]
                else:
                    print(detail["original_aliases"])

    amazon_tags[subclass]["low_quality"] = set(amazon_tags[subclass]["low_quality"])
    
            

In [64]:
for element in amazon_tags[subclass]["low_quality"]:
    if isinstance(element, dict):
        print("dict exists: ", element)

In [65]:
def standarizing_amazon(row):
    if row["extend_class"] == "色彩":
        row["standarized_tag"] = row["extend_detail"]
        return row
    else:
        standarized_tag = amazon_tags[row['extend_class']]['tags'].get(row['extend_detail'], "N/A")
        if standarized_tag == "N/A":
            if row["extend_detail"] in amazon_tags[row['extend_class']]['low_quality']:
                standarized_tag = "low_quality"

        row["standarized_tag"] = standarized_tag
        return row
    

In [66]:
tqdm.pandas(desc = "standarizing amazon products extend...")
amazon_standarized = amazon_products_extend.progress_apply(standarizing_amazon, axis = 1)

standarizing amazon products extend...: 100%|██████████| 1036016/1036016 [02:52<00:00, 6002.22it/s]


In [67]:
na_unique = len(amazon_standarized[amazon_standarized['standarized_tag'] == "N/A"]["extend_detail"].unique())
total_unique = len(amazon_standarized['extend_detail'].unique())
coverage_rate = (total_unique - na_unique) / total_unique
print(f"標籤種類覆蓋率: {coverage_rate:.2%}")

# 計算每個類別中，unique 標籤的覆蓋率
coverage_by_class_unique = amazon_standarized.groupby('extend_class').apply(
    lambda group: (group[group['standarized_tag'] != "N/A"]['extend_detail'].nunique()) / 
                  (group['extend_detail'].nunique()), include_groups = False
).sort_values(ascending=False)
coverage_by_class_unique

標籤種類覆蓋率: 95.49%


extend_class
色彩      1.000000
族群      0.998970
使用情境    0.995902
功能      0.993926
訴求      0.973241
風格      0.900812
材質      0.378425
dtype: float64

## Save

In [68]:
momo_standarized.to_excel("outputs/products_extend_momo_standarized.xlsx")

In [69]:
amazon_standarized.to_excel("outputs/products_extend_amazon_standarized.xlsx")

## Verification (by Gemini)

In [138]:
class Claude:

    batch_request_temp = lambda custom_id, system_prompt, input_msg: Request(
        custom_id = custom_id,
        params = MessageCreateParamsNonStreaming(
            model = "claude-3-7-sonnet-20250219",
            max_tokens = 20000,
            system = system_prompt,
            messages=[{
                "role": "user",
                "content": input_msg,
            }]
        ))
    
    user_prompt_temp = lambda id, product_categories_combination, tag_class, list_of_tags, list_of_low_quality_tags: json.dumps({
        "id": id,
        "product_categories_combination": product_categories_combination,
        "tag_class": tag_class,
        "list_of_tags": list_of_tags,
        "list_of_low_quality_tags": list_of_low_quality_tags
    }, ensure_ascii= False)
    
    def send_batch_requests(client, df, system_prompt):
        
        batch_items = []
        for _, row in df.iterrows():
            user_prompt = Claude.user_prompt_temp(
                str(row["id"]),
                f"{row["domain_from_classification"]}/{row["category_from_classification"]}/{row["subcategory_from_classification"]}/{row['further_subcategory_from_classification']}",
                str(row["extend_class"]),
                str(row["standarized_tag"]),
                str(row["low_quality_tag"])
            )
            request_obj = Claude.batch_request_temp(custom_id = row['id'],
                                        system_prompt = system_prompt,
                                        input_msg = user_prompt)
            
            batch_items.append(request_obj)

        # * Send Request
        batch = client.messages.batches.create(requests = batch_items)
        print(f"📦 Batch Created: batch_id: {batch.id}")
        return batch.id
    
    def retrieve_result(client, batch_id):
        request_status = client.messages.batches.retrieve(
                batch_id
            ).request_counts
        
        if request_status.processing > 0:
            print(f"this batch has not been fully processed. status: {request_status}")
        else:
            results = []
            decode_error_indices = []
            for result in client.messages.batches.results(
                batch_id
            ):
                primary_key = result.custom_id
                
                content     = LlmManager.find_json_object(result.result.message.content[0].text)
                if content == "DecodeError":
                    print(f"decode error at {primary_key}")
                    decode_error_indices.append(primary_key)
                    continue
                in_usage       = result.result.message.usage.input_tokens
                out_usage      = result.result.message.usage.output_tokens
                content["pk"] = primary_key
                content["usage_input"] = in_usage
                content["usage_output"] = out_usage
                results.append(content)

            if request_status.errored > 0:
                print(f"{request_status.errored} rows ended with errors")

            return (results, decode_error_indices)
        
    def calculate_usage(row):
        return (row["usage_input"] * 1.5 / 1000000 + row["usage_output"] * 7.5 / 1000000) * 30

### Momo

In [ ]:
momo_standarized = pd.read_excel("outputs/products_extend_momo_standarized.xlsx")

In [70]:
momo_standarized_toVerify = momo_standarized[~(momo_standarized['standarized_tag'] == "N/A")]

In [71]:
momo_standarized_toVerify.head(2)

,id,source_product_id,domain,category,subcategory,further_subcategory,extend_class,extend_subclass,extend_detail,extend_detail_raw,extend_unit,source,domain_from_classification,category_from_classification,subcategory_from_classification,further_subcategory_from_classification,match_key,standarized_tag
0,33c78847-7bd5-4a78-8139-9b9f461b4fb3,momo_10029797,NaN,NaN,NaN,NaN,使用情境,NaN,家庭,家庭,NaN,momo,智慧家電,通訊,行動通訊設備,無線電話,momo_10029797_使用情境_None_家庭_None,家庭使用
1,e9fbc893-9549-4d3e-a038-db38f5b0f283,momo_10029797,NaN,NaN,NaN,NaN,使用情境,NaN,辦公室,辦公室,NaN,momo,智慧家電,通訊,行動通訊設備,無線電話,momo_10029797_使用情境_None_辦公室_None,辦公


作法：
1. 篩出每個 extend class + 子品類組合 下面的 unique standardized tags
2. 問 AI，這些 tag 在特定子品類組合下面是否會過於廣泛或是太細節
3. 請 AI 給修正建議

Step 1.

In [72]:
momo_standarized_toVerify.loc[:, "low_quality_tag"] = (momo_standarized_toVerify
                                                .copy()
                                                .apply(
                                                    lambda row: row['extend_detail'] if row['standarized_tag'] == "low_quality" 
                                                    else "N/A", axis = 1))
momo_extend_grouped = (momo_standarized_toVerify
        .groupby(
            ["domain_from_classification", 
            "category_from_classification", 
            "subcategory_from_classification", 
            "further_subcategory_from_classification", 
            "extend_class"])
        .agg({"standarized_tag": lambda x: pd.Series([_ for _ in x if _ != "low_quality"]).unique().tolist(),
            "low_quality_tag": lambda x: pd.Series([_ for _ in x if _ != "N/A"]).unique().tolist()})
        .reset_index())


/tmp/ipykernel_5827/95274915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  momo_standarized_toVerify.loc[:, "low_quality_tag"] = (momo_standarized_toVerify


In [73]:
momo_extend_grouped["id"] = [f"momo_{i+1}" for i in range(len(momo_extend_grouped))]
momo_extend_grouped.head(3)

,domain_from_classification,category_from_classification,subcategory_from_classification,further_subcategory_from_classification,extend_class,standarized_tag,low_quality_tag,id
0,健康照護,健康與調理用品,循環/能量用品,健康頸鍊/手環,使用情境,"[身體按摩, 按摩舒緩, 大人使用, 工作效率, 水上活動, 全身使用, 穿戴使用, 運動訓...","[手部, 臀部, 足部, 骨盆, 下半身, 一整天都可以配戴, 浮潛, 氣場正向能量, 氣場...",momo_1
1,健康照護,健康與調理用品,循環/能量用品,健康頸鍊/手環,功能,"[促進血液循環, 消除疲勞, 減壓功能, 長時間使用, 補充, 提升睡眠品質, 壓力控制, ...","[消除肩頸硬叩叩, 增加深層睡眠, 提升身心平衡, 思路清晰, 增加深眠, 回應更靈敏, 易...",momo_2
2,健康照護,健康與調理用品,循環/能量用品,健康頸鍊/手環,族群,"[幼童, 男性, 女性, 成人, 年長者, 孕婦, 嬰幼兒, 青少年, 運動愛好者, 時尚愛...","[大人, 肩頸]",momo_3


In [75]:
sys_prompt = """You are a competent marketing analysis assistant. Now you are helping me verify the reasonability of the standarized tag for product extensive detail data from e-commerce platform.

I will give you a tag class, a list of tags under the tag class, and the corresponding product categories for these tags. I need you to focus on these lists of tags and to verify whether there is any tag that is too broadly defined, too narrowly defined, or unresonable or incomprehensive under the product category combination. 

The input looks like the following:
{
    "id": unique idenfication number for each request[str],
    "product_categories_combination": a string of product categories combination, in the following format: domain/category/subcategory/further_subcategory[str],
    "tag_class": targed tag class for all tags.[str],
    "list_of_tags": a list of tags under the specified tag_class with respect to the product_categories_combination[list],
    "list_of_low_quality_tags": a list of low quality tags under the specified tag_class with respect to the product_categories_combination[list]
}

CONTENT REQUIREMENTS:
- In terms of the "list_of_tags" column, ONLY answer tags that you think having problems, with reasoning and recommendation on a new name of the tag. The recommendation should be made with a view to make the tag more suitable for the specific categories combination.

- As for the "list_of_low_quality_tags" column, answer a polished tag name for each tag inside, with reasoning. You should strive in renaming the low quality tag into an existing tag in the 'list_of_tags' column.

The output schema is as follows:
{
    "id": output EXACTLY the same id as input[str],
    "list_of_tags_advices": [
        {
            "original_tag_name": the EXACT original tag name that you think having problems in the list_of_tags,
            "recommended_tag_name": your recommended tag name,
            "reasoning": reasoning
        },...
    ],
    "list_of_low_quality_tags_advices": [
        {
            "original_tag_name": the EXACT original tag name in the list_of_low_quality_tags,
            "recommended_tag_name": your recommended tag name. better be the existing tag name in the list_of_tags column,
            "reasoning": reasoning
        }
    ]
}

TECHNICAL REQUIREMENTS:
1. Use JSON as output format, and return only the JSON object. Do not answer any other texts.
2. Use double quote " for JSON format, but use single quote for any quotation mark inside a JSON string, to avoid JSON decode error.
3. Do not change the original tag name, so that I can map it back."""

In [83]:
config = dotenv_values('.env')
gemini = genai.Client(api_key = config["GEMINI_KEY"])
model_params = types.GenerateContentConfig(
    system_instruction = sys_prompt
)

In [84]:
momo_extend_grouped_no_color = momo_extend_grouped[momo_extend_grouped["extend_class"] != "色彩"]

Send Batch Request

In [85]:
claude_client = anthropic.Anthropic(
    api_key=config["CLAUDE_KEY"]
)
# * test on few data
Claude.send_batch_requests(claude_client, momo_extend_grouped_no_color, sys_prompt)

📦 Batch Created: batch_id: msgbatch_017psSBUggV5Gtw5n8G4jMoX


'msgbatch_017psSBUggV5Gtw5n8G4jMoX'

Get status & Save

In [140]:
claude_client.messages.batches.retrieve(
    'msgbatch_017psSBUggV5Gtw5n8G4jMoX'
).request_counts

res = Claude.retrieve_result(claude_client, 'msgbatch_017psSBUggV5Gtw5n8G4jMoX')
momo_succeeded_round1 = pd.DataFrame(res[0])
momo_decode_error_round1 = res[1]
try:
    momo_succeeded_round1["total_cost_twd"] = momo_succeeded_round1.apply(Claude.calculate_usage, axis = 1)

except:
    pass

decode error at momo_340
decode error at momo_430
decode error at momo_444
decode error at momo_451
decode error at momo_468
decode error at momo_945
decode error at momo_962
decode error at momo_1145
decode error at momo_1619
decode error at momo_1682
decode error at momo_1683
decode error at momo_1897
decode error at momo_2126


round 2 (重跑 decode error 的)

In [141]:
momo_decode_error_round1

['momo_340',
 'momo_430',
 'momo_444',
 'momo_451',
 'momo_468',
 'momo_945',
 'momo_962',
 'momo_1145',
 'momo_1619',
 'momo_1682',
 'momo_1683',
 'momo_1897',
 'momo_2126']

In [142]:
momo_extend_grouped_no_color_round2 = (momo_extend_grouped_no_color
                                       .set_index("id")
                                       .loc[momo_decode_error_round1, :]
                                       .reset_index())

In [131]:
claude_client = anthropic.Anthropic(
    api_key=config["CLAUDE_KEY"]
)
# * second trial
Claude.send_batch_requests(claude_client, momo_extend_grouped_no_color_round2, sys_prompt)

📦 Batch Created: batch_id: msgbatch_01KSGMUKbfVMj7bim9rU8i5s


'msgbatch_01KSGMUKbfVMj7bim9rU8i5s'

In [143]:
claude_client.messages.batches.retrieve(
    'msgbatch_01KSGMUKbfVMj7bim9rU8i5s'
).request_counts

res = Claude.retrieve_result(claude_client, 'msgbatch_01KSGMUKbfVMj7bim9rU8i5s')
momo_succeeded_round2 = pd.DataFrame(res[0])
momo_decode_error_round2 = res[1]
try:
    momo_succeeded_round2["total_cost_twd"] = momo_succeeded_round2.apply(Claude.calculate_usage, axis = 1)

except:
    pass

momo_succeeded_round2

decode error at momo_444
decode error at momo_451
decode error at momo_1682


,id,list_of_tags_advices,list_of_low_quality_tags_advices,pk,usage_input,usage_output,total_cost_twd
0,momo_340,"[{'original_tag_name': '模擬真實體驗', 'recommended_...","[{'original_tag_name': '智慧進水', 'recommended_ta...",momo_340,3423,2710,0.763785
1,momo_430,"[{'original_tag_name': '可調整設計', 'recommended_t...","[{'original_tag_name': '燈照角度可隨意調整', 'recommend...",momo_430,3643,1924,0.596835
2,momo_468,"[{'original_tag_name': '廣闊操作', 'recommended_ta...","[{'original_tag_name': '多種吹法', 'recommended_ta...",momo_468,5450,2857,0.888075
3,momo_945,"[{'original_tag_name': 'PC用戶', 'recommended_ta...","[{'original_tag_name': '具備藍牙 4.0 功能的設備使用者', 'r...",momo_945,4337,2376,0.729765
4,momo_962,"[{'original_tag_name': '可替換設計', 'recommended_t...","[{'original_tag_name': '快速液晶技術', 'recommended_...",momo_962,4243,2905,0.844560
5,momo_1145,"[{'original_tag_name': '超輕量', 'recommended_tag...","[{'original_tag_name': '流行美觀', 'recommended_ta...",momo_1145,3004,1579,0.490455
6,momo_1619,"[{'original_tag_name': '啟動功能', 'recommended_ta...","[{'original_tag_name': '拼出旋律', 'recommended_ta...",momo_1619,3327,2689,0.754740
7,momo_1683,"[{'original_tag_name': '卡通角色粉絲', 'recommended_...","[{'original_tag_name': 'STEIFF', 'recommended_...",momo_1683,1645,5415,1.292400
8,momo_1897,"[{'original_tag_name': '家居收納', 'recommended_ta...","[{'original_tag_name': '單邊擠奶', 'recommended_ta...",momo_1897,2940,3785,0.983925
9,momo_2126,"[{'original_tag_name': '輔助行動', 'recommended_ta...","[{'original_tag_name': '軟墊便圈', 'recommended_ta...",momo_2126,1027,2014,0.499365


round 3 (重跑 round 2 decode error 的)

In [144]:
momo_decode_error_round2

['momo_444', 'momo_451', 'momo_1682']

In [145]:
momo_extend_grouped_no_color_round3 = (momo_extend_grouped_no_color_round2
                                       .set_index("id")
                                       .loc[momo_decode_error_round2, :]
                                       .reset_index())

In [147]:
claude_client = anthropic.Anthropic(
    api_key=config["CLAUDE_KEY"]
)
# * second trial
Claude.send_batch_requests(claude_client, momo_extend_grouped_no_color_round3, sys_prompt)

📦 Batch Created: batch_id: msgbatch_01QgPm93enEDgNJwx68B1Wuf


'msgbatch_01QgPm93enEDgNJwx68B1Wuf'

In [154]:
claude_client.messages.batches.retrieve(
    'msgbatch_01QgPm93enEDgNJwx68B1Wuf'
).request_counts

res = Claude.retrieve_result(claude_client, 'msgbatch_01QgPm93enEDgNJwx68B1Wuf')
momo_succeeded_round3 = pd.DataFrame(res[0])
momo_decode_error_round3 = res[1]
try:
    momo_succeeded_round3["total_cost_twd"] = momo_succeeded_round3.apply(Claude.calculate_usage, axis = 1)

except:
    pass

momo_succeeded_round3

,id,list_of_tags_advices,list_of_low_quality_tags_advices,pk,usage_input,usage_output,total_cost_twd
0,momo_444,"[{'original_tag_name': '空氣淨化', 'recommended_ta...","[{'original_tag_name': '適用多型號', 'recommended_t...",momo_444,4538,1386,0.516060
1,momo_451,"[{'original_tag_name': '免插電', 'recommended_tag...","[{'original_tag_name': '無耗材', 'recommended_tag...",momo_451,3956,3491,0.963495
2,momo_1682,"[{'original_tag_name': '狗狗散步', 'recommended_ta...","[{'original_tag_name': '適合抓握', 'recommended_ta...",momo_1682,2411,2402,0.648945


In [ ]:
# 
# * Concatenate All Results
momo_verified_agg = pd.concat([momo_succeeded_round1, momo_succeeded_round2, momo_succeeded_round3])

In [161]:
momo_verified_agg.columns

Index(['id', 'list_of_tags_advices', 'list_of_low_quality_tags_advices', 'pk',
       'usage_input', 'usage_output', 'total_cost_twd'],
      dtype='object')

In [187]:
# * list of tags advices (一般 tag 的建議)
momo_ls_tags_advices = momo_verified_agg.explode('list_of_tags_advices').reset_index(drop = True)
momo_ls_tags_advices = pd.concat([momo_ls_tags_advices, 
                                  pd.json_normalize(momo_ls_tags_advices['list_of_tags_advices'])], axis = 1) 
momo_ls_tags_advices = momo_ls_tags_advices.drop(["list_of_low_quality_tags_advices", 'list_of_tags_advices'], axis = 1)
momo_ls_tags_advices['label'] = 'normal'

# * list of low quality tags 
momo_ls_low_quality_tags_advices = momo_verified_agg.explode('list_of_low_quality_tags_advices').reset_index(drop = True)
momo_ls_low_quality_tags_advices = pd.concat([momo_ls_low_quality_tags_advices, 
                                  pd.json_normalize(momo_ls_low_quality_tags_advices['list_of_low_quality_tags_advices'])], axis = 1) 
momo_ls_low_quality_tags_advices = momo_ls_low_quality_tags_advices.drop(["list_of_low_quality_tags_advices", 'list_of_tags_advices'], axis = 1)
momo_ls_low_quality_tags_advices['label'] = 'low_quality'


In [199]:
momo_verified_merged_back = (
    pd.merge(momo_extend_grouped_no_color, 
             pd.concat([momo_ls_tags_advices, momo_ls_low_quality_tags_advices]).sort_values(["id", "label"]),
             on = "id",
             how = "left")
    .drop(["standarized_tag", "low_quality_tag", "pk"], axis = 1))

In [201]:
momo_verified_merged_back.head(3)

,domain_from_classification,category_from_classification,subcategory_from_classification,further_subcategory_from_classification,extend_class,id,usage_input,usage_output,total_cost_twd,original_tag_name,recommended_tag_name,reasoning,label
0,健康照護,健康與調理用品,循環/能量用品,健康頸鍊/手環,使用情境,momo_1,907,1538,0.386865,手部,穿戴使用,手環是穿戴在手部的產品，可歸類為穿戴使用的更一般性標籤,low_quality
1,健康照護,健康與調理用品,循環/能量用品,健康頸鍊/手環,使用情境,momo_1,907,1538,0.386865,臀部,全身使用,健康頸鍊/手環通常不針對臀部，其效果通常宣稱影響全身,low_quality
2,健康照護,健康與調理用品,循環/能量用品,健康頸鍊/手環,使用情境,momo_1,907,1538,0.386865,足部,全身使用,健康頸鍊/手環通常不針對足部，應歸類為全身使用,low_quality


In [202]:
momo_verified_merged_back.to_excel("outputs/momo_tags_recommendation_0715.xlsx")

### Amazon

In [4]:
amazon_standarized = pd.read_excel("outputs/products_extend_amazon_standarized.xlsx")

In [134]:
amazon_standarized_toVerify = amazon_standarized[~(amazon_standarized['standarized_tag'] == "N/A")]

In [135]:
amazon_standarized_toVerify.loc[:, "low_quality_tag"] = (amazon_standarized_toVerify
                                                .copy()
                                                .apply(
                                                    lambda row: row['extend_detail'] if row['standarized_tag'] == "low_quality" 
                                                    else "N/A", axis = 1))
amazon_extend_grouped = (amazon_standarized_toVerify
        .groupby(
            ["domain_from_classification", 
            "category_from_classification", 
            "subcategory_from_classification", 
            "further_subcategory_from_classification", 
            "extend_class"])
        .agg({"standarized_tag": lambda x: pd.Series([_ for _ in x if _ != "low_quality"]).unique().tolist(),
            "low_quality_tag": lambda x: pd.Series([_ for _ in x if _ != "N/A"]).unique().tolist()})
        .reset_index())


/tmp/ipykernel_5827/1728745840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amazon_standarized_toVerify.loc[:, "low_quality_tag"] = (amazon_standarized_toVerify


In [136]:
amazon_extend_grouped["id"] = [f"amazon_{i+1}" for i in range(len(amazon_extend_grouped))]
amazon_extend_grouped.head(3)

,domain_from_classification,category_from_classification,subcategory_from_classification,further_subcategory_from_classification,extend_class,standarized_tag,low_quality_tag,id
0,健康照護,健康與調理用品,循環/能量用品,冷熱水袋/敷墊,使用情境,"[Muscle Pain Relief, Specialized Equipment Use...","[Safe to use during pregnancy, Cryotherapy, Im...",amazon_1
1,健康照護,健康與調理用品,循環/能量用品,冷熱水袋/敷墊,功能,"[Pain Management, Physical Support, Regulation...","[Increase fertility, Provide prenatal vitamins...",amazon_2
2,健康照護,健康與調理用品,循環/能量用品,冷熱水袋/敷墊,族群,"[Healthcare Professionals, Health Condition Su...","[Individuals with eyelash extensions, Shift wo...",amazon_3


In [137]:
amazon_extend_grouped_no_color = amazon_extend_grouped[amazon_extend_grouped["extend_class"] != "色彩"]

Send batch requests

In [152]:
claude_client = anthropic.Anthropic(
    api_key=config["CLAUDE_KEY"]
)
# * full data request
Claude.send_batch_requests(claude_client, amazon_extend_grouped_no_color, sys_prompt)

📦 Batch Created: batch_id: msgbatch_011ZU1Xy3Gf6pW1r8HVNVMPs


'msgbatch_011ZU1Xy3Gf6pW1r8HVNVMPs'

In [205]:
claude_client.messages.batches.retrieve(
    'msgbatch_011ZU1Xy3Gf6pW1r8HVNVMPs'
).request_counts

res = Claude.retrieve_result(claude_client, 'msgbatch_011ZU1Xy3Gf6pW1r8HVNVMPs')
amazon_succeeded_round1 = pd.DataFrame(res[0])
amazon_decode_error_round1 = res[1]
try:
    amazon_succeeded_round1["total_cost_twd"] = amazon_succeeded_round1.apply(Claude.calculate_usage, axis = 1)

except:
    pass

amazon_succeeded_round1

decode error at amazon_505
decode error at amazon_635
decode error at amazon_714


,id,list_of_tags_advices,list_of_low_quality_tags_advices,pk,usage_input,usage_output,total_cost_twd
0,amazon_1,"[{'original_tag_name': 'Medical Supplies', 're...",[{'original_tag_name': 'Safe to use during pre...,amazon_1,2217,4464,1.104165
1,amazon_2,"[{'original_tag_name': 'Cooking Mode', 'recomm...","[{'original_tag_name': 'Increase fertility', '...",amazon_2,3247,2040,0.605115
2,amazon_3,[{'original_tag_name': 'Healthcare Professiona...,[{'original_tag_name': 'Individuals with eyela...,amazon_3,1049,1077,0.289530
3,amazon_4,"[{'original_tag_name': 'Purified Water', 'reco...","[{'original_tag_name': 'ILEX', 'recommended_ta...",amazon_4,1487,3107,0.765990
4,amazon_6,"[{'original_tag_name': 'Hypoallergenic', 'reco...","[{'original_tag_name': 'Non-medicinal, safe wi...",amazon_6,4544,2811,0.836955
...,...,...,...,...,...,...,...
1883,amazon_2201,"[{'original_tag_name': 'Protection', 'recommen...",[{'original_tag_name': 'Keeps child cool with ...,amazon_2201,1825,3411,0.849600
1884,amazon_2202,"[{'original_tag_name': 'Adults', 'recommended_...","[{'original_tag_name': 'Gift buyers', 'recomme...",amazon_2202,834,519,0.154305
1885,amazon_2203,"[{'original_tag_name': 'Leather-Like', 'recomm...","[{'original_tag_name': 'special Velcro', 'reco...",amazon_2203,1035,1106,0.295425
1886,amazon_2205,"[{'original_tag_name': 'Built-in Features', 'r...",[{'original_tag_name': 'pioneering safety and ...,amazon_2205,2271,3336,0.852795


In [206]:
amazon_extend_grouped_no_color_round2 = (amazon_extend_grouped_no_color
                                       .set_index("id")
                                       .loc[amazon_decode_error_round1, :]
                                       .reset_index())

In [208]:
claude_client = anthropic.Anthropic(
    api_key=config["CLAUDE_KEY"]
)
# * second_trial
Claude.send_batch_requests(claude_client, amazon_extend_grouped_no_color_round2, sys_prompt)

📦 Batch Created: batch_id: msgbatch_01QoudarXnAtkGSaaKpf6K8i


'msgbatch_01QoudarXnAtkGSaaKpf6K8i'

In [209]:
claude_client.messages.batches.retrieve(
    'msgbatch_01QoudarXnAtkGSaaKpf6K8i'
).request_counts

res = Claude.retrieve_result(claude_client, 'msgbatch_01QoudarXnAtkGSaaKpf6K8i')
amazon_succeeded_round2 = pd.DataFrame(res[0])
amazon_decode_error_round2 = res[1]
try:
    amazon_succeeded_round2["total_cost_twd"] = amazon_succeeded_round2.apply(Claude.calculate_usage, axis = 1)

except:
    pass

amazon_succeeded_round2

decode error at amazon_635


,id,list_of_tags_advices,list_of_low_quality_tags_advices,pk,usage_input,usage_output,total_cost_twd
0,amazon_505,"[{'original_tag_name': 'Cooking Mode', 'recomm...",[{'original_tag_name': 'Apply soft eye compres...,amazon_505,3822,8294,2.038140
1,amazon_714,"[{'original_tag_name': 'Protection & Safety', ...",[{'original_tag_name': 'Protect Magicband from...,amazon_714,2819,2260,0.635355


In [210]:
amazon_extend_grouped_no_color_round3 = (amazon_extend_grouped_no_color
                                       .set_index("id")
                                       .loc[amazon_decode_error_round2, :]
                                       .reset_index())

In [212]:
claude_client = anthropic.Anthropic(
    api_key=config["CLAUDE_KEY"]
)
# * second_trial
Claude.send_batch_requests(claude_client, amazon_extend_grouped_no_color_round3, sys_prompt)

📦 Batch Created: batch_id: msgbatch_01A2yXqUvGkFZjx9uxmE43M7


'msgbatch_01A2yXqUvGkFZjx9uxmE43M7'

In [213]:
claude_client.messages.batches.retrieve(
    'msgbatch_01A2yXqUvGkFZjx9uxmE43M7'
).request_counts

res = Claude.retrieve_result(claude_client, 'msgbatch_01A2yXqUvGkFZjx9uxmE43M7')
amazon_succeeded_round3 = pd.DataFrame(res[0])
amazon_decode_error_round3 = res[1]
try:
    amazon_succeeded_round3["total_cost_twd"] = amazon_succeeded_round3.apply(Claude.calculate_usage, axis = 1)

except:
    pass

amazon_succeeded_round3

,id,list_of_tags_advices,list_of_low_quality_tags_advices,pk,usage_input,usage_output,total_cost_twd
0,amazon_635,"[{'original_tag_name': 'Cyberpunk Style', 'rec...","[{'original_tag_name': 'Hot Dog Toaster Red', ...",amazon_635,936,1633,0.409545


In [214]:
# 
# * Concatenate All Results
amazon_verified_agg = pd.concat([amazon_succeeded_round1, amazon_succeeded_round2, amazon_succeeded_round3])

In [215]:
# * list of tags advices (一般 tag 的建議)
amazon_ls_tags_advices = amazon_verified_agg.explode('list_of_tags_advices').reset_index(drop = True)
amazon_ls_tags_advices = pd.concat([amazon_ls_tags_advices, 
                                  pd.json_normalize(amazon_ls_tags_advices['list_of_tags_advices'])], axis = 1) 
amazon_ls_tags_advices = amazon_ls_tags_advices.drop(["list_of_low_quality_tags_advices", 'list_of_tags_advices'], axis = 1)
amazon_ls_tags_advices['label'] = 'normal'

# * list of low quality tags 
amazon_ls_low_quality_tags_advices = amazon_verified_agg.explode('list_of_low_quality_tags_advices').reset_index(drop = True)
amazon_ls_low_quality_tags_advices = pd.concat([amazon_ls_low_quality_tags_advices, 
                                  pd.json_normalize(amazon_ls_low_quality_tags_advices['list_of_low_quality_tags_advices'])], axis = 1) 
amazon_ls_low_quality_tags_advices = amazon_ls_low_quality_tags_advices.drop(["list_of_low_quality_tags_advices", 'list_of_tags_advices'], axis = 1)
amazon_ls_low_quality_tags_advices['label'] = 'low_quality'


In [216]:
amazon_verified_merged_back = (
    pd.merge(amazon_extend_grouped_no_color, 
             pd.concat([amazon_ls_tags_advices, amazon_ls_low_quality_tags_advices]).sort_values(["id", "label"]),
             on = "id",
             how = "left")
    .drop(["standarized_tag", "low_quality_tag", "pk"], axis = 1))

In [220]:
amazon_verified_merged_back.to_excel("outputs/amazon_tags_recommendation_0715.xlsx")

## Reclassify low quality tags using verified data

### Momo

In [ ]:
momo_standarized_copy = momo_standarized.copy()
momo_standarized_copy["original_tag_name"] = momo_standarized_copy.apply(lambda row: row["extend_detail"] if row["standarized_tag"] == "low_quality"
                                                                           else row["standarized_tag"], axis = 1)

In [236]:
momo_standarized_copy.head(3)

,id,source_product_id,domain,category,subcategory,further_subcategory,extend_class,extend_subclass,extend_detail,extend_detail_raw,extend_unit,source,domain_from_classification,category_from_classification,subcategory_from_classification,further_subcategory_from_classification,match_key,standarized_tag,original_tag_name
0,33c78847-7bd5-4a78-8139-9b9f461b4fb3,momo_10029797,NaN,NaN,NaN,NaN,使用情境,NaN,家庭,家庭,NaN,momo,智慧家電,通訊,行動通訊設備,無線電話,momo_10029797_使用情境_None_家庭_None,家庭使用,家庭使用
1,e9fbc893-9549-4d3e-a038-db38f5b0f283,momo_10029797,NaN,NaN,NaN,NaN,使用情境,NaN,辦公室,辦公室,NaN,momo,智慧家電,通訊,行動通訊設備,無線電話,momo_10029797_使用情境_None_辦公室_None,辦公,辦公
2,04973514-0a3c-4a96-8685-6c24983a9f65,momo_10278628,NaN,NaN,NaN,NaN,使用情境,NaN,辦公室,辦公室,NaN,momo,智慧家電,通訊,行動通訊設備,無線電話,momo_10278628_使用情境_None_辦公室_None,辦公,辦公


In [246]:
#
# * Scripts of reclassify start here:
momo_full_merged = (pd.merge(momo_standarized_copy,
                    momo_verified_merged_back.drop(["id", "usage_input", "usage_output", "total_cost_twd"], axis = 1),
                    on = ["domain_from_classification", "category_from_classification", "subcategory_from_classification", "further_subcategory_from_classification", "extend_class", "original_tag_name"],
                    how = "left"))
momo_full_merged_dropped_NA = momo_full_merged.drop(momo_full_merged[momo_full_merged["standarized_tag"] == "N/A"].index)
momo_full_merged_dropped_NA

In [255]:
# * 若沒有 recommendation，則用原本的
momo_full_merged_dropped_NA["recommended_tag_name"] = momo_full_merged_dropped_NA.apply(
    lambda row: row["recommended_tag_name"] if str(row["recommended_tag_name"]) != "nan" else row["original_tag_name"],
    axis = 1
)
momo_full_merged_dropped_NA["label"] = momo_full_merged_dropped_NA["label"].fillna("normal")

In [256]:
momo_full_merged_dropped_NA["final_refined_tag"] = momo_full_merged_dropped_NA.apply(
    lambda row: row["original_tag_name"] if row['label'] == "normal" 
    else row["recommended_tag_name"],
    axis = 1
)

In [265]:
print(f"unique tag counts before reclassify: {len(momo_standarized_copy["original_tag_name"].unique()) - 1}")
print(f"unique tag counts after reclassify:  {len(momo_full_merged_dropped_NA["final_refined_tag"].unique())}")

unique tag counts before reclassify: 57305
unique tag counts after reclassify:  32605


In [277]:
(momo_full_merged_dropped_NA
 .drop(["domain", "category", "subcategory", "further_subcategory", "extend_detail", "standarized_tag", "original_tag_name", "recommended_tag_name", "reasoning", "label"], axis = 1)
 .rename(columns = {
     "domain_from_classification": "domain",
     "category_from_classification": "category",
     "subcategory_from_classification": "subcategory",
     "further_subcategory_from_classification": "further_subcategory",
     "final_refined_tag": "extend_detail"
 })
 .to_excel("outputs/products_extend_momo_standardized_verified_reclassified.xlsx"))

In [275]:
momo_full_merged_dropped_NA.columns

Index(['id', 'source_product_id', 'domain', 'category', 'subcategory',
       'further_subcategory', 'extend_class', 'extend_subclass',
       'extend_detail', 'extend_detail_raw', 'extend_unit', 'source',
       'domain_from_classification', 'category_from_classification',
       'subcategory_from_classification',
       'further_subcategory_from_classification', 'match_key',
       'standarized_tag', 'original_tag_name', 'recommended_tag_name',
       'reasoning', 'label', 'final_refined_tag'],
      dtype='object')

### Amazon

In [267]:
amazon_standarized_copy = amazon_standarized.copy()
amazon_standarized_copy["original_tag_name"] = amazon_standarized_copy.apply(lambda row: row["extend_detail"] if row["standarized_tag"] == "low_quality"
                                                                           else row["standarized_tag"], axis = 1)
amazon_standarized_copy.head(3)

,id,source_product_id,domain,category,subcategory,further_subcategory,extend_class,extend_subclass,extend_detail,extend_detail_raw,extend_unit,source,domain_from_classification,category_from_classification,subcategory_from_classification,further_subcategory_from_classification,project_year,match_key,standarized_tag,original_tag_name
0,262eee05-b0af-4ec2-a711-d2cce5727c18,amazon_0545498562,NaN,NaN,NaN,NaN,訴求,NaN,award-winning craft kit for kids,award-winning craft kit for kids,NaN,amazon,母嬰兒童,兒童玩具,益智玩具,串珠/拼豆,113,amazon_0545498562_訴求_None_award-winning craft ...,low_quality,award-winning craft kit for kids
1,a2f645d4-4340-46b4-9e69-58710e15ffec,amazon_0545498562,NaN,NaN,NaN,NaN,訴求,NaN,includes 9 vibrant clay colors,includes 9 vibrant clay colors,NaN,amazon,母嬰兒童,兒童玩具,益智玩具,串珠/拼豆,113,amazon_0545498562_訴求_None_includes 9 vibrant c...,low_quality,includes 9 vibrant clay colors
2,090cccc4-8132-438c-865c-8a16b44678ce,amazon_0545498562,NaN,NaN,NaN,NaN,訴求,NaN,simple step-by-step instructions included,simple step-by-step instructions included,NaN,amazon,母嬰兒童,兒童玩具,益智玩具,串珠/拼豆,113,amazon_0545498562_訴求_None_simple step-by-step ...,Easy To Use,Easy To Use


In [269]:
#
# * Scripts of reclassify start here:
amazon_full_merged = (pd.merge(amazon_standarized_copy,
                    amazon_verified_merged_back.drop(["id", "usage_input", "usage_output", "total_cost_twd"], axis = 1),
                    on = ["domain_from_classification", "category_from_classification", "subcategory_from_classification", "further_subcategory_from_classification", "extend_class", "original_tag_name"],
                    how = "left"))
amazon_full_merged_dropped_NA = amazon_full_merged.drop(amazon_full_merged[amazon_full_merged["standarized_tag"] == "N/A"].index)
amazon_full_merged_dropped_NA.head(3)

,id,source_product_id,domain,category,subcategory,further_subcategory,extend_class,extend_subclass,extend_detail,extend_detail_raw,...,category_from_classification,subcategory_from_classification,further_subcategory_from_classification,project_year,match_key,standarized_tag,original_tag_name,recommended_tag_name,reasoning,label
0,262eee05-b0af-4ec2-a711-d2cce5727c18,amazon_0545498562,NaN,NaN,NaN,NaN,訴求,NaN,award-winning craft kit for kids,award-winning craft kit for kids,...,兒童玩具,益智玩具,串珠/拼豆,113,amazon_0545498562_訴求_None_award-winning craft ...,low_quality,award-winning craft kit for kids,Award-Winning Design,This low-quality tag is too specific and wordy...,low_quality
1,a2f645d4-4340-46b4-9e69-58710e15ffec,amazon_0545498562,NaN,NaN,NaN,NaN,訴求,NaN,includes 9 vibrant clay colors,includes 9 vibrant clay colors,...,兒童玩具,益智玩具,串珠/拼豆,113,amazon_0545498562_訴求_None_includes 9 vibrant c...,low_quality,includes 9 vibrant clay colors,Color Variety,This tag is too specific about the exact numbe...,low_quality
2,090cccc4-8132-438c-865c-8a16b44678ce,amazon_0545498562,NaN,NaN,NaN,NaN,訴求,NaN,simple step-by-step instructions included,simple step-by-step instructions included,...,兒童玩具,益智玩具,串珠/拼豆,113,amazon_0545498562_訴求_None_simple step-by-step ...,Easy To Use,Easy To Use,NaN,NaN,NaN


In [270]:
# * 若沒有 recommendation，則用原本的
amazon_full_merged_dropped_NA["recommended_tag_name"] = amazon_full_merged_dropped_NA.apply(
    lambda row: row["recommended_tag_name"] if str(row["recommended_tag_name"]) != "nan" else row["original_tag_name"],
    axis = 1
)
amazon_full_merged_dropped_NA["label"] = amazon_full_merged_dropped_NA["label"].fillna("normal")

In [271]:
amazon_full_merged_dropped_NA["final_refined_tag"] = amazon_full_merged_dropped_NA.apply(
    lambda row: row["original_tag_name"] if row['label'] == "normal" 
    else row["recommended_tag_name"],
    axis = 1
)

In [272]:
print(f"unique tag counts before reclassify: {len(amazon_standarized_copy["original_tag_name"].unique()) - 1}")
print(f"unique tag counts after reclassify:  {len(amazon_full_merged_dropped_NA["final_refined_tag"].unique())}")

unique tag counts before reclassify: 98704
unique tag counts after reclassify:  69716


In [278]:
(amazon_full_merged_dropped_NA
 .drop(["domain", "category", "subcategory", "further_subcategory", "extend_detail", "standarized_tag", "original_tag_name", "recommended_tag_name", "reasoning", "label"], axis = 1)
 .rename(columns = {
     "domain_from_classification": "domain",
     "category_from_classification": "category",
     "subcategory_from_classification": "subcategory",
     "further_subcategory_from_classification": "further_subcategory",
     "final_refined_tag": "extend_detail"
 })
 .to_excel("outputs/products_extend_amazon_standardized_verified_reclassified.xlsx"))

### 臨時任務：0715
1. Product Extend 與數字型垂直合併
2. Product Extend 做兩個版本：前端拿掉與不拿掉（不拿掉的就是我這邊產出的，拿掉的 = 與 product 拿掉的 df Left Merge）

Amazon

In [288]:
products_extend_amazon_numerical = pd.read_excel("sources/amazon_products_extend_數字標準化_0715.xlsx")
products_to_filter = pd.read_excel("sources/products_amazon_0715_delete.xlsx")[["source_product_id"]]
products_extend_full = pd.concat([amazon_full_merged_dropped_NA.drop(["domain", "category", "subcategory", "further_subcategory", "extend_detail", "standarized_tag", "original_tag_name", "recommended_tag_name", "reasoning", "label"], axis = 1)
                                    .rename(columns = {
                                        "domain_from_classification": "domain",
                                        "category_from_classification": "category",
                                        "subcategory_from_classification": "subcategory",
                                        "further_subcategory_from_classification": "further_subcategory",
                                        "final_refined_tag": "extend_detail"
                                    }),
                                  products_extend_amazon_numerical])
products_extend_filter = pd.merge(products_to_filter, products_extend_full, how = 'left', on = 'source_product_id')

In [291]:
products_extend_full.to_csv("outputs/products_extend_amazon_0715.csv")
products_extend_filter.to_csv("outputs/products_extend_amazon_0715_delete.csv")

Momo

In [292]:
products_extend_momo_numerical = pd.read_excel("sources/momo_products_extend_數字標準化_0715.xlsx")
products_to_filter_momo = pd.read_excel("sources/products_momo_0715_delete.xlsx")[["source_product_id"]]
products_extend_full_momo = pd.concat([momo_full_merged_dropped_NA.drop(["domain", "category", "subcategory", "further_subcategory", "extend_detail", "standarized_tag", "original_tag_name", "recommended_tag_name", "reasoning", "label"], axis = 1)
                                    .rename(columns = {
                                        "domain_from_classification": "domain",
                                        "category_from_classification": "category",
                                        "subcategory_from_classification": "subcategory",
                                        "further_subcategory_from_classification": "further_subcategory",
                                        "final_refined_tag": "extend_detail"
                                    }),
                                  products_extend_momo_numerical])
products_extend_filter_momo = pd.merge(products_to_filter_momo, products_extend_full_momo, how = 'left', on = 'source_product_id')

In [295]:
products_extend_full_momo.to_csv("outputs/products_extend_momo_0715.csv")
products_extend_filter_momo.to_csv("outputs/products_extend_momo_0715_delete.csv")

### 0721
Product Extend with 3 columns (raw, reclassified tag, recommended tag)

Amazon

In [25]:
# * amazon
standardized_amazon = pd.read_excel("outputs/products_extend_amazon_standarized.xlsx")
recommended_amazon  = pd.read_excel("outputs/products_extend_amazon_standardized_verified_reclassified.xlsx")

In [26]:
amazon_merged = pd.merge(
        standardized_amazon.drop(
            [
                "extend_detail",
                "domain", 
                "category",
                "subcategory",
                "further_subcategory",
            ],
            axis = 1
        ),
        recommended_amazon.drop(
                    [
            "id",
            "source_product_id", 
            "extend_class",
            "extend_subclass",
            "extend_detail_raw",
            "extend_unit",
            "project_year",
            "source",
            "Unnamed: 0"
            ], axis = 1
        ),
        how = 'inner',
        on = 'match_key'
    )

In [27]:
amazon_merged = (
    amazon_merged[[
    "id",
    "match_key",
    "source_product_id",
    'category', 
    'subcategory',
    'further_subcategory',
    'extend_class',
    'extend_subclass',
    'extend_detail_raw',
    'standarized_tag',
    'extend_detail',
    'project_year',
    'source',
    'extend_unit']]
    .rename(columns = {
        'extend_detail_raw': "extend_detail_origin",
        "standarized_tag": "standardized_detail",
        "extend_detail": "recommended_detail"}
    ))

In [28]:
amazon_merged.to_excel("outputs/product_extend_amazon_with_history_0721.xlsx")

Momo

In [30]:
# * momo
standardized_momo = pd.read_excel("outputs/products_extend_momo_standarized.xlsx")
recommended_momo  = pd.read_excel("outputs/products_extend_momo_standardized_verified_reclassified.xlsx")

momo_merged = pd.merge(
        standardized_momo.drop(
            [
                "extend_detail",
                "domain", 
                "category",
                "subcategory",
                "further_subcategory",
            ],
            axis = 1
        ),
        recommended_momo.drop(
                    [
            "id",
            "source_product_id", 
            "extend_class",
            "extend_subclass",
            "extend_detail_raw",
            "extend_unit",
            "source",
            "Unnamed: 0"
            ], axis = 1
        ),
        how = 'inner',
        on = 'match_key'
    )

momo_merged = (
    momo_merged[[
    "id",
    "match_key",
    "source_product_id",
    'category', 
    'subcategory',
    'further_subcategory',
    'extend_class',
    'extend_subclass',
    'extend_detail_raw',
    'standarized_tag',
    'extend_detail',
    'source',
    'extend_unit']]
    .rename(columns = {
        'extend_detail_raw': "extend_detail_origin",
        "standarized_tag": "standardized_detail",
        "extend_detail": "recommended_detail"}
    ))

momo_merged.to_excel("outputs/product_extend_momo_with_history_0721.xlsx")